In [2]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [3]:
import csv

csv_file = 'data.csv'
question = []   
answer = []
issue = []
os = []
with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)

    # Iterate over each row in the CSV file
    for row in reader:
        question.append(row[0])
        answer.append(row[1])
        issue.append(row[2])
        os.append(row[3])

from langchain_core.documents import Document
docs = []

for i in range(0,len(question)):
    docs.append(Document(
        page_content=f"question:{question[i]}, answer: {answer[i]}",
        metadata={"issue":issue[i], "os": os[i]}
        )
    )



In [4]:
from langchain_community.vectorstores import Qdrant


vectorstore = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="it_sol_chatbot",
)

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768",groq_api_key="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")

In [6]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="issue",
        description="The issue in the computer",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="os",
        description="The operating system in the computer",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="solution",
        description="the solution for the issue",
        type="string or list[string]",
    ),
   
]
document_content_description = "Brief description of the issue in computer"


retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    k=1
)

In [13]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
from langchain.retrievers.self_query.qdrant import QdrantTranslator
prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser


from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain.retrievers.self_query.qdrant import QdrantTranslator
retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=QdrantTranslator(metadata_key="metadata"),
    k=1
)


In [17]:
data = retriever.invoke("My computer is overheating then usual")

print(data)

[Document(page_content='question:Computer overheating when turned on, answer: Clean the dust from vents and ensure proper airflow', metadata={'issue': 'computer overheating', 'os': 'windows', '_id': 'dc08cea380d542eda34ee160ca5ba5c4', '_collection_name': 'it_sol_chatbot'}), Document(page_content='question:Overheating during gaming, answer: Improve cooling system and reduce graphic settings', metadata={'issue': 'gaming overheating', 'os': 'windows', '_id': '2e70ab658f3e40759d0e3b261f71ab03', '_collection_name': 'it_sol_chatbot'}), Document(page_content="question:Error 'Device overheating', answer: Clean vents and improve ventilation around device", metadata={'issue': 'device overheating', 'os': 'windows', '_id': '024e60623088486f99e7f0ff7edea9cb', '_collection_name': 'it_sol_chatbot'}), Document(page_content='question:Computer freezes randomly, answer: Check for overheating and update device drivers', metadata={'issue': 'freezing', 'os': 'windows', '_id': 'f2cac4c0b59843f8aada47a525f2b8

In [11]:
from langchain.prompts import ChatPromptTemplate
import re
print("Hi I am your IT bot just ask me any queries if you face any in your ")

while True:
    question = input("Enter your question: ")
    if question == "":
        break
    data = retriever.invoke(question)

    match = re.match(r'question:(.*?),\s*answer:(.*)', data[0].page_content, re.IGNORECASE)
    if match:
      question = match.group(1).strip()
      answer = match.group(2).strip()
    

    # Define the ChatPromptTemplate
    system = "You are a helpful IT support assistant."
    human = "{text}"
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

    chain = prompt | llm
    ans = chain.invoke({"text": f"""This is the question asked {question} and this is the answer to it found in database {answer} Please show a formatted answer for this inabout 30 words"""}).content
    print("Suggestion:  "+ans+"\n\n")


Hi I am your IT bot just ask me any queries if you face any in your device


Enter your question:  My computer is overheating when i turn it on


Suggestion:  To resolve computer overheating, clean dust from vents and ensure proper airflow. This will help maintain an optimal temperature for your device.




Enter your question:  My computer is running slower then usual


Suggestion:  To improve slow performance after startup, try disabling unnecessary startup programs and running disk defragmentation. These steps can help optimize your system's speed and efficiency.




Enter your question:  There is some error in disk read and write


Suggestion:  To resolve a disk read/write error, check your disk's health using the CHKDSK tool and back up important data as a precaution. Act promptly to prevent data loss.




Enter your question:  I am getting a error 'High CPU usage'


Suggestion:  To resolve high CPU usage, identify and close unnecessary processes consuming high levels of CPU resources. Regularly monitor and optimize running processes to maintain optimal performance.




Enter your question:  
